In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!sudo apt update

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Check this site for the latest download link
# https://www.apache.org/dyn/closer.lua/spark
!wget -q https://dlcdn.apache.org/spark/spark-3.5.2/spark-3.5.2-bin-hadoop3.tgz
!tar xf spark-3.5.2-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,107 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [59.5 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
# Create Spark Session in localhost
spark = SparkSession.builder.master("local").\
    appName("ALS-Recommendation").\
    getOrCreate()

# Set spark context
sc = spark.sparkContext
sc.setLogLevel("ERROR")

movies_df = spark.read.text("/content/sample_movielens_ratings.txt")

movies_df = movies_df.withColumn("userID", split(movies_df["value"], "::").getItem(0)) \
.withColumn("movieID", split(movies_df["value"], "::").getItem(1)) \
.withColumn("rating", split(movies_df["value"], "::").getItem(2)) \
.withColumn("timestamp", split(movies_df["value"], "::").getItem(3))

movies_df = movies_df.drop("value")
movies_df = movies_df.drop("timestamp")

movies_df = movies_df.withColumn("userID", movies_df["userID"].cast("int"))
movies_df = movies_df.withColumn("movieID", movies_df["movieID"].cast("int"))
movies_df = movies_df.withColumn("rating", movies_df["rating"].cast("int"))

movies_df.show(n=10, truncate=False)

+------+-------+------+
|userID|movieID|rating|
+------+-------+------+
|0     |2      |3     |
|0     |3      |1     |
|0     |5      |2     |
|0     |9      |4     |
|0     |11     |1     |
|0     |12     |2     |
|0     |15     |1     |
|0     |17     |1     |
|0     |19     |1     |
|0     |21     |1     |
+------+-------+------+
only showing top 10 rows



In [13]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

als = ALS(maxIter=10, regParam=0.01, rank=5, userCol="userID",
itemCol="movieID", ratingCol="rating")

model = als.fit(movies_df)

# Generate top 10 movie recommendations for each user
userRecs = model.recommendForAllUsers(numItems=3)

In [14]:
predictions = model.transform(movies_df)
predictions.show(truncate=False)

evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)

+------+-------+------+----------+
|userID|movieID|rating|prediction|
+------+-------+------+----------+
|0     |2      |3     |2.219122  |
|0     |3      |1     |1.0306422 |
|0     |5      |2     |1.6931715 |
|0     |9      |4     |1.57683   |
|0     |11     |1     |1.6796913 |
|0     |12     |2     |1.9960976 |
|0     |15     |1     |0.9491459 |
|0     |17     |1     |0.9589144 |
|0     |19     |1     |1.4507687 |
|0     |21     |1     |1.3571905 |
|0     |23     |1     |1.141135  |
|0     |26     |3     |1.7177043 |
|0     |27     |1     |0.4107145 |
|0     |28     |1     |1.7104793 |
|0     |29     |1     |2.0318964 |
|0     |30     |1     |0.74834096|
|0     |31     |1     |1.7076833 |
|0     |34     |1     |1.1356139 |
|0     |37     |1     |1.4148008 |
|0     |41     |2     |2.3270774 |
+------+-------+------+----------+
only showing top 20 rows

